In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import tacotron2
import malaya_speech.config
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [4]:
tacotron2_config = malaya_speech.config.tacotron2_config

In [5]:
model = tacotron2.Model(tacotron2.Config(vocab_size = 20, **tacotron2_config), training = True)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
input_ids = tf.placeholder(tf.int32, [None, None])
input_lengths = tf.placeholder(tf.int32, [None])
speaker_ids = tf.constant([0], dtype = tf.int32)
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])
mel_actuals = tf.placeholder(tf.float32, [None, None, 80])

In [7]:
guided = tacotron2.generate_guided_attention(mel_lengths[0], input_lengths[0])
guided = tf.expand_dims(guided, 0)
guided

<tf.Tensor 'ExpandDims:0' shape=(1, ?, ?) dtype=float32>

In [8]:
r = model(input_ids, input_lengths, speaker_ids, mel_outputs, mel_lengths, training = True)
r

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(<tf.Tensor 'model/Reshape:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'model/add:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'model/Reshape_1:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'model/transpose:0' shape=(?, ?, ?) dtype=float32>)

In [9]:
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss

In [10]:
binary_crossentropy = tf.keras.losses.BinaryCrossentropy(
    from_logits=True, reduction='none'
)
mse = tf.keras.losses.MeanSquaredError(
    reduction='none'
)
mae = tf.keras.losses.MeanAbsoluteError(
    reduction='none'
)
decoder_output, post_mel_outputs, stop_token_predictions, alignment_histories = r
mel_loss_before = calculate_3d_loss(
    mel_actuals, decoder_output, loss_fn=mae
)
mel_loss_after = calculate_3d_loss(
    mel_actuals, post_mel_outputs, loss_fn=mae
)
max_mel_length = tf.reduce_max(mel_lengths)
stop_gts = tf.expand_dims(
    tf.range(tf.reduce_max(max_mel_length), dtype=tf.int32), 0
)
stop_gts = tf.tile(
    stop_gts, [tf.shape(mel_lengths)[0], 1]
)
stop_gts = tf.cast(
    tf.math.greater_equal(stop_gts, tf.expand_dims(mel_lengths, 1)),
    tf.float32,
)
stop_token_loss = calculate_2d_loss(
    stop_gts, stop_token_predictions, loss_fn=binary_crossentropy
)
attention_masks = tf.cast(
    tf.math.not_equal(guided, -1.0), tf.float32
)
loss_att = tf.reduce_sum(
    tf.abs(alignment_histories * guided) * attention_masks,
    axis=[1, 2],
)
loss_att /= tf.reduce_sum(attention_masks, axis=[1, 2])

In [11]:
loss = stop_token_loss + mel_loss_before + mel_loss_after + loss_att

In [12]:
# r = model.inference(input_ids, input_lengths, speaker_ids)
# r

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [14]:
sess.run(r, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]})

(array([[[-0.06796341, -0.0804487 , -0.04431556, ..., -0.09433724,
          -0.02345449,  0.08772649],
         [-0.04318153, -0.08245155, -0.05823935, ..., -0.11458633,
          -0.02950949,  0.05547573],
         [-0.01866408, -0.08206702, -0.08863966, ..., -0.13753544,
          -0.03806493,  0.04022491],
         ...,
         [-0.06704423, -0.09294236, -0.07631406, ..., -0.13891862,
           0.02630183,  0.09242541],
         [-0.06643584, -0.08368027, -0.07834595, ..., -0.14442576,
           0.01060861,  0.09623958],
         [-0.07137919, -0.07196188, -0.08635556, ..., -0.1341814 ,
           0.00101672,  0.08422293]]], dtype=float32),
 array([[[-1.5531547 ,  0.5537216 ,  1.0227194 , ...,  0.07452253,
           1.4796675 ,  2.0808184 ],
         [ 1.3986566 ,  0.8528912 , -0.09732963, ...,  1.044044  ,
          -1.1566794 ,  0.28264594],
         [ 0.5521731 ,  2.7178888 ,  0.638061  , ...,  0.71136516,
          -0.50971717, -0.05817204],
         ...,
         [ 3.75097

In [15]:
sess.run(loss, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_actuals: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]})

array([[3.021349 , 2.9683666, 2.8150268, 3.0592651, 3.1796381, 3.131577 ,
        3.1811478, 2.8446221, 3.1216266, 3.0640397, 3.1034694, 3.2381985,
        2.7253952, 2.7224984, 2.9553146, 2.9229646, 3.131834 , 2.9843235,
        2.9755402, 2.7743883, 2.8919098, 3.168142 , 3.0310464, 3.0300584,
        2.908257 , 2.8068876, 2.9689984, 3.1704912, 3.035336 , 2.8225808,
        2.9783864, 3.1301632, 2.971221 , 2.9498286, 3.4484158, 2.9097295,
        2.7904568, 3.1013207, 2.825247 , 2.9891722, 3.097128 , 2.9827783,
        2.8775887, 3.1963136, 3.2869973, 2.872067 , 2.916367 , 2.865512 ,
        3.0789337, 2.6820312]], dtype=float32)